# Dry snow density

The dry snow density is utilized in 2 places to calculate the maximum amount of water that can be held in the snowpack. The first is in `SnowState.init_layers()` when a new snowpack is generated. The equation to determine the dry snow density (`pysnobal.core.snow.dry_snow_density`) is:

```python
(rho - sat * RHO_W0) / (1 - sat * RHO_W0 / RHO_ICE)
```

where `rho` is the snow density, `sat` is the liquid water content as volume ratio. This was based on equations in Colbeck 1978 and Davis et al, 1985. However, in `runoff` it uses a simpler equation

```python
(m_s - h2o_total) / z_s
```

The `h2o_total` can be greater than `h2o_max` or the maximum water content of the snowpack (`h2o_sat` > 1). Modifying the `runoff` to use the `dry_snow_density` changes the model results slightly. The main difference is the `h2o` of the snowpack and `swi` timing. However, the changes are fairly minimal and <1% for `h2o` and `swi`

In [ ]:
import os
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from inicheck.tools import MasterConfig, cast_all_variables, get_user_config

import pysnobal
from pysnobal.pysnobal import PySnobal
from pysnobal.core.snow import dry_snow_density

def nse(obs, model):
    num = np.sum((model - obs)**2)
    denom = np.sum((obs - np.mean(obs))**2)
    return 1 - num/denom

In [ ]:
# Dry snow density from `dry_snow_density`
h2o_vol = np.arange(0, 0.05, 0.0005)
rho = np.arange(100, 600, 10)

[RHO, H2O] = np.meshgrid(rho, h2o_vol)
d_rho = dry_snow_density(RHO, H2O)


plt.imshow(
    RHO - d_rho,
    extent=[np.min(rho), np.max(rho), np.min(h2o_vol), np.max(h2o_vol)],
    aspect='auto')
plt.title('Density minus dry snow density')
plt.xlabel('Density')
plt.ylabel('h2o_vol')
plt.colorbar()


In [ ]:
# run for the 25 years against the original dry snow density model in runoff

wys = range(1984, 2009)
out_dir = '../pysnobal/tests/test_data_point/gold_csv/'
config_file = os.path.join(out_dir, 'config.ini')

master_config = os.path.join(Path(pysnobal.__file__).parent, 'pysnobal_core_config.ini')
mcfg = MasterConfig(path=master_config)
config = get_user_config(config_file, mcfg=mcfg)


for wy in wys:
    print(wy)

    # Update the config
    config.raw_cfg['time'].update({
        'start_date': '10-01-{} 00:00'.format(wy-1),
        'end_date': '09-30-{} 23:00'.format(wy)
    })
    config.raw_cfg['files'].update({
        'input_csv': os.path.join('25year', 'wy{}.input.csv'.format(wy)),
        'output_csv': os.path.join('25year', 'dry_snow.wy{}.output.csv'.format(wy))
    })

    config.apply_recipes()
    config = cast_all_variables(config, config.mcfg)

    # run with updated config
    status = PySnobal(config).run()

In [ ]:
# Sanity check that we're outputing the correct stuff
df = None
for idx,wy in enumerate(wys):

    start_date = pd.to_datetime('10-01-{} 01:00'.format(wy-1))
    end_date = pd.to_datetime('09-30-{} 23:00'.format(wy))
    
    file_name = os.path.join(out_dir, '25year', 'wy{}.output.csv'.format(wy))
    gold = pd.read_csv(file_name, index_col='date_time', parse_dates=True)

    file_name = os.path.join(out_dir, '25year', 'dry_snow.wy{}.output.csv'.format(wy))
    new = pd.read_csv(file_name, index_col='date_time', parse_dates=True)

    d = gold - new
    dd = d.describe()

    for col in gold.columns:
        dd.loc['NSE', col] = nse(new[col], gold[col])

    dd['water_year'] = wy
    dd.reset_index(inplace=True)
    dd.rename(columns={'index': 'metric'}, inplace=True)

    if df is None:
        df = dd
    else:
        df = pd.concat([df, dd])

    
df

In [ ]:
df.reset_index(inplace=True)
df.set_index(['metric', 'water_year'], inplace=True)
if 'index' in df.columns:
    df.drop(columns='index', inplace=True)

fig, ax = plt.subplots(5, 5, figsize=(25, 25))
ax = ax.flatten()

for idx, col in enumerate(df.columns):
    df2 = df.loc[['min', '25%', '50%', '75%', 'max'], col]
    df2.index.rename([col, 'water_year'], inplace=True)
    df2.unstack(level=0).plot(ax=ax[idx])

plt.savefig('figures/dry_snow_density.png')
